<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/text_classification/char_rnn_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import required libraries 
from io import open 
import glob 
import os

import torch 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available")
else:
  print("CUDA is not available")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

### Load and preprocess

In [ ]:
import string 
import unicodedata 

# all ascii characters including space, dot, comma, semicolon, single quote
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

all_letters

In [ ]:
def unicodeToAscii(s):
  """ 
    Convert unicode string to ascii string
  """
  ascii_str = ""
  for ch in unicodedata.normalize('NFD', s):
    if unicodedata.category(ch) != "Mn":
      if ch in all_letters:
        ascii_str += ch
  
  return ascii_str

print(unicodeToAscii('Ślusàrski'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = "/content/drive/MyDrive/data/names/*.txt"

def findFiles(path): 
  return glob.glob(path)

findFiles(data_path)

In [ ]:
language_names = {}
all_languages = []

total_names = 0

for filename in findFiles(data_path):
    language = os.path.splitext(os.path.basename(filename))[0]
    all_languages.append(language)
    read_names = open(filename, encoding='utf-8').read().strip().split('\n')
    names = [unicodeToAscii(name) for name in read_names]
    language_names[language] = names
    total_names += len(names)

n_languages = len(all_languages)

In [ ]:
all_languages

In [ ]:
print(language_names['Italian'][:10])

In [ ]:
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][all_letters.find(letter)] = 1
    return tensor

print(letterToTensor('a'))

In [ ]:
def nameToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

jones_tensor = nameToTensor('Jones')

print(jones_tensor.size())
print(jones_tensor)

### Model

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 256
model = RNN(n_letters, n_hidden, n_languages)

print(model)

### Train

In [ ]:
# define some helper functions

import random

def languageFromOutput(output):
  top_n, top_i = output.topk(1)
  language_i = top_i[0].item()
  return all_languages[language_i], language_i

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    language = randomChoice(all_languages)

    random_language_names = language_names[language]
    name = randomChoice(random_language_names)

    language_tensor = torch.tensor([all_languages.index(language)], dtype=torch.long)
    name_tensor = nameToTensor(name)

    return language, name, language_tensor, name_tensor

In [ ]:
def train(model, language_tensor, name_tensor, criterion, lr):
    hidden = model.initHidden()

    model.zero_grad()
    for i in range(name_tensor.size()[0]):
        output, hidden = model(name_tensor[i], hidden)

    loss = criterion(output, language_tensor)
    loss.backward()

    for p in model.parameters():
        p.data.add_(p.grad.data, alpha=-lr)

    return output, loss.item()

In [ ]:
criterion = nn.NLLLoss()
lr = 0.005

num_epochs = 200000
current_loss = 0 
all_losses = []
print_every = 10000
plot_every = 1000

for epoch in range(1, num_epochs + 1):
    language, name, language_tensor, name_tensor = randomTrainingExample()

    output, loss = train(model, language_tensor, name_tensor, criterion, lr)

    current_loss += loss

    if epoch % print_every == 0:
        guess, _ = languageFromOutput(output)
        correct = '✓' if guess == language else '✗ (%s)' % language

        print('%d %d%% %.4f %s | %s %s' % (epoch,
                                           epoch / num_epochs * 100,
                                           loss, 
                                           name, 
                                           guess,
                                           correct))

    if epoch % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

In [ ]:
plt.figure()
plt.plot(all_losses)
plt.show()

### Test

In [ ]:
def test(name):
  with torch.no_grad():
    hidden = model.initHidden()
    name_tensor = nameToTensor(name)
    for i in range(name_tensor.size()[0]):
      output, hidden = model(name_tensor[i], hidden)
  return output

In [ ]:
def prediction(name, n_predictions):
  predictions = []

  output = test(name)
  topv, topi = output.topk(n_predictions, 1, True)

  for i in range(n_predictions):
    value = topv[0][i].item()
    language_index = topi[0][i].item()
    preds = all_languages[language_index]
    print('(%.2f) %s' % (value, preds))

    predictions.append([value, preds])

In [ ]:
prediction('Dovesky', 5)